In [1]:
import numpy as np

import torch
import torch.nn as nn
    
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import pytorch_lightning as pl
from torch.utils.data import DataLoader

from pathlib import Path
from typing import List, Tuple

import numpy as np
import pandas as pd

import torch
import pytorch_lightning as pl
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import plotly.graph_objects as go
from statsmodels.tsa.stattools import adfuller
import math
import functools

import sys
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

# TNC SSL Training 

In [2]:
data_path = Path('data/TNC/HAR_data')
x_train = np.load(data_path/'x_train.npy')
y_train = np.load(data_path/'y_train.npy')
x_test = np.load(data_path/'x_test.npy')
y_test = np.load(data_path/'y_test.npy')

x_train.shape, y_train.shape, x_test.shape, y_test.shape

((21, 561, 281), (21, 281), (9, 561, 288), (9, 288))

In [3]:
class TNCDataset(Dataset):
    def __init__(
        self,
        data: np.ndarray,
        mc_sample_size: int,
        window_size: int,
        state: float = None,
        adf: bool = True,
        augmentation: int = 1  # Simple repeat the vecvor 'augmentation' times
    ):
        super().__init__()
        self.time_series = data
        self.T = data.shape[-1]
        self.window_size = window_size
        self.sliding_gap = int(window_size * 25.2)
        self.window_per_sample = (self.T - 2 * self.window_size) // self.sliding_gap
        self.mc_sample_size = mc_sample_size
        self.state = state
        self.adf = adf
        self.epsilon = None
        self.augmentation = augmentation

    def __len__(self):
        return len(self.time_series) * self.augmentation

    # @functools.cache
    def __getitem__(self, ind):
        ind = ind % len(self.time_series)       # To repeat augmentation
        t = np.random.randint(2 * self.window_size, self.T - 2 * self.window_size)
        x_t = self.time_series[ind][
            :, t - self.window_size // 2 : t + self.window_size // 2
        ]
        X_close = self._find_neighours(self.time_series[ind], t)
        X_distant = self._find_non_neighours(self.time_series[ind], t)

        if self.state is None:
            y_t = -1
        else:
            y_t = np.round(
                np.mean(
                    self.state[ind][
                        t - self.window_size // 2 : t + self.window_size // 2
                    ]
                )
            )
        return (
            x_t.astype(np.float32),
            X_close.astype(np.float32),
            X_distant.astype(np.float32),
            y_t,
        )

    def _find_neighours(self, x, t):
        T = self.time_series.shape[-1]

        # ---- Do the ADF test ----
        gap = self.window_size
        corr = []
        for w_t in range(self.window_size, 4 * self.window_size, gap):
            try:
                p_val = 0
                for f in range(x.shape[-2]):
                    p = adfuller(
                        np.array(
                            x[
                                f, max(0, t - w_t) : min(x.shape[-1], t + w_t)
                            ].reshape(
                                -1,
                            )
                        )
                    )[1]
                    p_val += 0.01 if math.isnan(p) else p
                corr.append(p_val / x.shape[-2])
            except:
                corr.append(0.6)
        self.epsilon = (
            len(corr)
            if len(np.where(np.array(corr) >= 0.01)[0]) == 0
            else (np.where(np.array(corr) >= 0.01)[0][0] + 1)
        )
        self.delta = 5 * self.epsilon * self.window_size
        # --------------------------

        ## Random from a Gaussian
        t_p = [
            int(t + np.random.randn() * self.epsilon * self.window_size)
            for _ in range(self.mc_sample_size)
        ]
        t_p = [
            max(self.window_size // 2 + 1, min(t_pp, T - self.window_size // 2))
            for t_pp in t_p
        ]
        x_p = np.stack(
            [
                x[:, t_ind - self.window_size // 2 : t_ind + self.window_size // 2]
                for t_ind in t_p
            ]
        )
        return x_p

    def _find_non_neighours(self, x, t):
        T = self.time_series.shape[-1]
        if t > T / 2:
            t_n = np.random.randint(
                self.window_size // 2,
                max((t - self.delta + 1), self.window_size // 2 + 1),
                self.mc_sample_size,
            )
        else:
            t_n = np.random.randint(
                min((t + self.delta), (T - self.window_size - 1)),
                (T - self.window_size // 2),
                self.mc_sample_size,
            )
        x_n = np.stack(
            [
                x[:, t_ind - self.window_size // 2 : t_ind + self.window_size // 2]
                for t_ind in t_n
            ]
        )

        if len(x_n) == 0:
            rand_t = np.random.randint(0, self.window_size // 5)
            if t > T / 2:
                x_n = x[:, rand_t : rand_t + self.window_size].unsqueeze(0)
            else:
                x_n = x[:, T - rand_t - self.window_size : T - rand_t].unsqueeze(0)
        return x_n

In [4]:
class Discriminator(torch.nn.Module):
    def __init__(self, input_size, device: str = "cpu"):
        super(Discriminator, self).__init__()
        self.input_size = input_size

        self.model = torch.nn.Sequential(
            torch.nn.Linear(2 * self.input_size, 4 * self.input_size),
            torch.nn.ReLU(inplace=True),
            torch.nn.Dropout(0.5),
            torch.nn.Linear(4 * self.input_size, 1),
        ).to(device)

        torch.nn.init.xavier_uniform_(self.model[0].weight)
        torch.nn.init.xavier_uniform_(self.model[3].weight)

    def forward(self, x, x_tild):
        """
        Predict the probability of the two inputs belonging to the same neighbourhood.
        """
        x_all = torch.cat([x, x_tild], -1)
        p = self.model(x_all)
        return p.view((-1,))

### Enconder

The code can be made considerably simpler when implementing.

In [5]:
class GRUEncoder(torch.nn.Module):
    def __init__(
        self,
        hidden_size: int = 100,
        in_channel: int = 561,
        encoding_size: int = 10,
        num_layers: int = 1,
        dropout: float = 0.0,
        bidirectional: bool = True,
        device: str = "cpu",
    ):
        super().__init__()
        self.hidden_size = hidden_size
        self.in_channel = in_channel
        self.num_layers = num_layers
        self.encoding_size = encoding_size
        self.bidirectional = bidirectional
        self.device = device
        
        self.rnn = torch.nn.GRU(
            input_size=self.in_channel,
            hidden_size=self.hidden_size,
            num_layers=num_layers,
            batch_first=False,
            dropout=dropout,
            bidirectional=bidirectional,
        ).to(device)

        self.nn = torch.nn.Linear(
            self.hidden_size * (int(self.bidirectional) + 1), self.encoding_size
        ).to(device)

    def forward(self, x):
        x = x.permute(2, 0, 1)

        past = torch.zeros(
            self.num_layers * (int(self.bidirectional) + 1),
            x.shape[1],
            self.hidden_size,
            device=self.device,
        )

        out, _ = self.rnn(
            x, past
        )  # out shape = [seq_len, batch_size, num_directions*hidden_size]
        encodings = self.nn(out[-1].squeeze(0))
        return encodings

### TNC Dataset

* Augmented Dickey-Fuller test:
    The neighborhood of a window should represent similar samples, so the range (represented by $\eta$) needs to identify the approximate time span within which the signal remains stationary, and the generative process does not change. For this purpose, Augmented Dickey-Fuller (ADF) is used to determine this region for every window.

    Determining a proper value for $\eta$ is part of the TNC framework.
    
    Video explaining Augmented Dickey-Fuller test: https://www.youtube.com/watch?v=1opjnegd_hA

In [6]:
class TNC(pl.LightningModule):
    def __init__(
        self,
        encoder: torch.nn.Module,
        discriminator: torch.nn.Module,
        mc_sample_size: int = 20,
        window_size: int = 4,
        w: float = 0.05,
        lr: float = 1e-3,
        weight_decay: float = 1e-5,
    ):
        super().__init__()
        self.encoder = encoder.to(self.device)
        self.discriminator = discriminator.to(self.device)
        self.mc_sample_size = mc_sample_size
        self.window_size = window_size
        self.w = w
        self.learning_rate = lr
        self.weight_decay = weight_decay
        self.loss_func = torch.nn.BCEWithLogitsLoss()
        self.training_step_losses = []

    def training_step(self, batch, batch_idx):
        x_t, x_p, x_n, _ = batch
        mc_sample = x_p.shape[1]
        batch_size, f_size, len_size = x_t.shape
        x_p = x_p.view(-1, f_size, len_size)
        x_n = x_n.view(-1, f_size, len_size)
        x_t = x_t.repeat(mc_sample, 1, 1)
        neighbors = torch.ones(len(x_p)).to(self.device)
        non_neighbors = torch.zeros(len(x_n)).to(self.device)
        x_t, x_p, x_n = x_t.to(self.device), x_p.to(self.device), x_n.to(self.device)

        z_t = self.encoder(x_t)
        z_p = self.encoder(x_p)
        z_n = self.encoder(x_n)

        d_p = self.discriminator(z_t, z_p)
        d_n = self.discriminator(z_t, z_n)

        p_loss = self.loss_func(d_p, neighbors)
        n_loss = self.loss_func(d_n, non_neighbors)
        n_loss_u = self.loss_func(d_n, neighbors)
        loss = (p_loss + self.w * n_loss_u + (1 - self.w) * n_loss) / 2

        self.training_step_losses.append(loss)
        return loss
    
    def on_train_epoch_end(self) -> None:
        # do something with all training_step outputs, for example:
        epoch_mean = torch.stack(self.training_step_losses).mean()
        self.log("train_loss", epoch_mean, on_epoch=True, on_step=False, prog_bar=True, logger=True)
        # free up the memory
        self.training_step_losses.clear()

    def configure_optimizers(self):
        learnable_parameters = list(self.discriminator.parameters()) + list(
            self.encoder.parameters()
        )

        optimizer = torch.optim.Adam(
            learnable_parameters, lr=self.learning_rate, weight_decay=self.weight_decay
        )
        return optimizer

In [7]:
class SimpleDataset:
    def __init__(self, X, y=None):
        self.X = X
        self.y = y
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        if self.y is not None:
            return self.X[idx].astype(np.float32), self.y[idx].astype(np.float32)
        else:
            return self.X[idx].astype(np.float32)
        
# train_dataset = SimpleDataset(x_train)
# test_dataset = SimpleDataset(x_test, y_test)
# len(train_dataset), len(test_dataset)

In [8]:
encoding_size = 10
window_size = 4
augmentation = 5
batch_size = 10
mc_sample_size = 20
workers = 10

In [9]:
# train test split X_train and y_train to train and validation using sklearn train_test_split
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

train_dataset = TNCDataset(X_train, mc_sample_size=20, window_size=4, augmentation=augmentation)
validation_dataset = TNCDataset(X_val, mc_sample_size=20, window_size=4, augmentation=augmentation)

len(train_dataset), len(validation_dataset)

(80, 25)

In [10]:
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, num_workers=workers, shuffle=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=batch_size, num_workers=workers, shuffle=True)

In [11]:
# Declaring the model
discriminator = Discriminator(input_size=encoding_size, device='cuda')
encoder = GRUEncoder(encoding_size=encoding_size, device='cuda')
tnc_model = TNC(
    encoder=encoder,
    discriminator=discriminator,
    window_size=window_size,
    mc_sample_size=mc_sample_size,
)

In [12]:
trainer = pl.Trainer(max_epochs=150, accelerator="gpu", devices=1)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [13]:
trainer.fit(tnc_model, train_dataloader, validation_dataloader)

You are using a CUDA device ('NVIDIA A100 80GB PCIe') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]

  | Name          | Type              | Params
----------------------------------------------------
0 | encoder       | GRUEncoder        | 399 K 
1 | discriminator | Discriminator     | 881   
2 | loss_func     | BCEWithLogitsLoss | 0     
----------------------------------------------------
400 K     Trainable params
0         Non-trainable params
400 K     Total params
1.603     Total estimated model params size (MB)


Epoch 149: 100%|██████████| 8/8 [00:22<00:00,  2.80s/it, v_num=24, train_loss=0.690]

`Trainer.fit` stopped: `max_epochs=150` reached.


Epoch 149: 100%|██████████| 8/8 [00:22<00:00,  2.83s/it, v_num=24, train_loss=0.690]


# TNC Fine-tunning

We are going to use the TNC model on the downstream task of classification. We will use same dataset and re-use the same encoder

In [126]:
from typing import Any, Optional

from torchmetrics.functional import accuracy


class StateClassifier(torch.nn.Module):
    def __init__(self, input_size: int = 10, n_classes: int = 6):
        super(StateClassifier, self).__init__()
        self.input_size = input_size
        self.n_classes = n_classes
        self.normalize = torch.nn.BatchNorm1d(self.input_size)
        self.nn = torch.nn.Linear(self.input_size, self.n_classes)
        torch.nn.init.xavier_uniform_(self.nn.weight)

    def forward(self, x):
        x = self.normalize(x)
        logits = self.nn(x)
        return logits
    
class TNC_Classifier(pl.LightningModule):
    def __init__(
        self,
        encoder: torch.nn.Module,
        classifier: torch.nn.Module,
        lr: float = 1e-2,
        weight_decay: float = 0.0,
        task_class: str = "multiclass",
        num_classes: int = 6
    ):
        super().__init__()
        self.encoder = encoder.to(self.device)
        self.classifier = classifier.to(self.device)
        self.learning_rate = lr
        self.weight_decay = weight_decay
        self.training_step_losses = []
        self.validation_step_losses = []
        self.loss_function = torch.nn.CrossEntropyLoss()
        self.task_class = task_class
        self.num_classes = num_classes
        
    def configure_optimizers(self) -> Any:
        optimizer = torch.optim.Adam(
            self.classifier.parameters(), lr=self.learning_rate, weight_decay=self.weight_decay
        )
        return optimizer
    
    def forward(self, x):
        encodings = self.encoder(x)
        predictions = self.classifier(encodings)
        return predictions
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        predictions = self.forward(x)
        loss = self.loss_function(predictions, y.long())
        self.training_step_losses.append(loss)
        return loss
    
    def on_train_epoch_end(self) -> None:
        # do something with all training_step outputs, for example:
        epoch_mean = torch.stack(self.training_step_losses).mean()
        self.log("train_loss", epoch_mean, on_epoch=True, on_step=False, prog_bar=True, logger=True)
        # free up the memory
        self.training_step_losses.clear()
    
    def validation_step(self, batch, batch_idx):
        loss, acc = self._shared_eval_step(batch, batch_idx)
        self.validation_step_losses.append(loss)
        metrics = {"val_acc": acc, "val_loss": loss}
        self.log_dict(metrics)
        return metrics
    
    def test_step(self, batch, batch_idx):
        loss, acc = self._shared_eval_step(batch, batch_idx)
        metrics = {"test_acc": acc, "test_loss": loss}
        self.log_dict(metrics)
        return metrics

        
    def on_validation_epoch_end(self) -> None:
        # do something with all training_step outputs, for example:
        epoch_mean = torch.stack(self.validation_step_losses).mean()
        self.log("val_loss", epoch_mean, on_epoch=True, on_step=False, prog_bar=True, logger=True)
        # free up the memory
        self.validation_step_losses.clear()

    def _shared_eval_step(self, batch, batch_idx):
        x, y = batch
        predictions = self.forward(x)
        loss = self.loss_function(predictions, y.long())
        acc = accuracy(torch.argmax(predictions, dim=1), y.long(), task=self.task_class, num_classes=self.num_classes)
        return loss, acc

In [127]:
data_path = Path('data/TNC/HAR_data')
x_train = np.load(data_path/'x_train.npy')
y_train = np.load(data_path/'y_train.npy')
x_test = np.load(data_path/'x_test.npy')
y_test = np.load(data_path/'y_test.npy')

x_train.shape, y_train.shape, x_test.shape, y_test.shape

((21, 561, 281), (21, 281), (9, 561, 288), (9, 288))

In [128]:
# This code as taken from the authors

def create_simulated_dataset(
    X_train, Y_train, X_test, Y_test, window_size=50, batch_size=100
):
    n_train = int(0.8 * len(X_train))
    n_valid = len(X_train) - n_train
    n_test = len(X_test)
    x_train, y_train = X_train[:n_train], Y_train[:n_train]
    x_valid, y_valid = X_train[n_train:], Y_train[n_train:]
    x_test, y_test = X_test, Y_test

    datasets = []
    for x, y, size in [
        (x_train, y_train, n_train),
        (x_test, y_test, n_test),
        (x_valid, y_valid, n_valid),
    ]:
        T = x.shape[-1]
        windows = np.split(
            x[:, :, : window_size * (T // window_size)], (T // window_size), -1
        )
        windows = np.concatenate(windows, 0)
        labels = np.split(
            y[:, : window_size * (T // window_size)], (T // window_size), -1
        )
        labels = np.round(np.mean(np.concatenate(labels, 0), -1))
        dset = SimpleDataset(windows, labels)
        datasets.append(dset)

    trainset, testset, validset = datasets[0], datasets[1], datasets[2]
    train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True)
    valid_loader = DataLoader(validset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(testset, batch_size=batch_size, shuffle=True)

    return train_loader, valid_loader, test_loader

In [144]:
train_loader, valid_loader, test_loader = create_simulated_dataset(x_train, y_train, x_test, y_test, window_size=4, batch_size=100)

In [145]:
encoding_size = 10
n_classes = 6

classifier = StateClassifier(input_size=encoding_size, n_classes=n_classes)
tnc_classifier = TNC_Classifier(encoder, classifier)

In [146]:
trainer = pl.Trainer(max_epochs=50, accelerator="gpu", devices=1)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [147]:
trainer.fit(tnc_classifier, train_dataloaders=train_loader, val_dataloaders=valid_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]

  | Name          | Type             | Params
---------------------------------------------------
0 | encoder       | GRUEncoder       | 399 K 
1 | classifier    | StateClassifier  | 86    
2 | loss_function | CrossEntropyLoss | 0     
---------------------------------------------------
399 K     Trainable params
0         Non-trainable params
399 K     Total params
1.600     Total estimated model params size (MB)


Epoch 0:   8%|▊         | 1/12 [00:00<00:00, 35.18it/s, v_num=48]          

Epoch 49: 100%|██████████| 12/12 [00:00<00:00, 19.40it/s, v_num=48, val_loss=0.958, train_loss=1.020]

`Trainer.fit` stopped: `max_epochs=50` reached.


Epoch 49: 100%|██████████| 12/12 [00:00<00:00, 13.59it/s, v_num=48, val_loss=0.958, train_loss=1.020]


## Classification evaluation

In [148]:
trainer.test(tnc_classifier, test_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]


Testing DataLoader 0: 100%|██████████| 7/7 [00:00<00:00, 39.51it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.49074074625968933    │
│         test_loss         │    0.9984065890312195     │
└───────────────────────────┴───────────────────────────┘

[{'test_acc': 0.49074074625968933, 'test_loss': 0.9984065890312195}]